In [19]:
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
from bokeh.layouts import  column, row
from bokeh.palettes import Category20_18
import re
import glob, os
from astropy.io import fits
from urllib import urlretrieve
from pyraf import iraf
import matplotlib.pyplot as plt
output_notebook()

Loading BokehJS ...

In [2]:
listafile = '../listCV.txt'
with open(listafile,'r') as f:
    for lines in f:
        star = lines.split(',')[0]
        url= lines.split(',')[1]
        name = url.split('/')[-1]
        if not os.path.isfile(name.strip()):
            urlretrieve(url,name.strip())
        #print(url)

# Plotting model Bokeh

Can plot the data or the datamodel changing in the code the 

```python
y = dataob['model']
```
or 
```python
y = dataob['flux']
```

In [3]:
#plot parameters
colors = Category20_18 #+ Viridis8 + Dark2_8 + Paired8 + Set1_4


xr = (5000,8900)
yr = (0,20)
p =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=1000
             )

hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
p.add_tools(hover2)


#os.chdir("/mydir")
for index,file in enumerate(glob.glob("spec*.fits")):
    ob = fits.open(file)
    dataob = ob[1].data
    
    x=10**(dataob['loglam'])
    y=dataob['model'] # or flux
    
    #If you want to have the name and not the spectra type change legend=name
    t = p.line(x=x,y=y,color=colors[index], line_alpha=1.0, 
           line_width=4,legend=ob[2].data['SUBCLASS'][0]+str(index),muted_alpha=0.,muted_color=colors[index])

    
    
    
p.legend.location = "top_left"
p.legend.click_policy="mute"
show(p)

# Table to IRAF 

In [4]:
for index,filename in enumerate(glob.glob("spec*.fits")):
    #import fits
    ob = fits.open(filename)
    #Get data and save wavelenft in x, and flux on y
    dataob = ob[1].data
    x = 10**(dataob['loglam'])
    y = dataob['model']

    #save to a .txt file
    namenewfits = '{name}.txt'.format(name=filename.split('.')[0])
    with open(namenewfits,'w') as file:
        for i in zip(x,y):
            file.write('{}\t{}\n'.format(i[0],i[1]))
    #Create fits file from text file interpolation. 
    iraffitsname = 'iraf'+filename.split('.')[0]
    if os.path.exists(iraffitsname+'.fits'):
        os.remove(iraffitsname+'.fits')

    iraf.noao.onedspec()
    iraf.dataio()
    iraf.rspectext(namenewfits,iraffitsname,dtype='interp')

In [5]:
ls

CV.ipynb                       spec-0266-51602-0051.fits
fited.log                      spec-0266-51602-0051.txt
irafspec-0266-51602-0051.fits  spec-0266-51630-0057.fits
irafspec-0266-51630-0057.fits  spec-0266-51630-0057.txt
irafspec-0266-51630-0530.fits  spec-0266-51630-0530.fits
irafspec-0268-51633-0129.fits  spec-0266-51630-0530.txt
lines.lines                    spec-0268-51633-0129.fits
login.cl                       spec-0268-51633-0129.txt
pyraf/                         uparm/


# Fit a gaussian

In [10]:
#Parametest for the fit:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx, array[idx]

def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
    
def gaussian(x, mu, sig,core):
    return core*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))




line = [8543,8598.]
regionfit = [[8520,8560],[8582,8609]]

#Initialize files log and lines
! echo '' > fited.log

for lines in zip(line,regionfit):
    regionf = "{} {}".format(lines[1][0],lines[1][1])
    #wavelenght
    xlimns = [find_nearest(x,i)[0] for i in lines[1]    ]
    wavex = x[xlimns[0]:xlimns[1]]
    lineszero = lines[0]
    ! echo '$lineszero' > lines.lines
    
    
for index,filename in enumerate(glob.glob("iraf*.fits")):

        iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                      fitbackground= 'yes', 
                      logfile='fited.log')
                      #,nerrsample='100',sigma0='4',invgain='4')


        #Plotting the gaussian
        #Find in log file  
        npattern = re.compile('[-\d.]+')
        gparameters=[]
        with open('fited.log','r') as file:
            for lines in file:
                if '(' not in lines:
                    temp = npattern.findall(lines)
                if len(temp) == 7 and all(isDigit(i) for i in temp):
                    gparameters.append(temp)

        #gaussian
        gparamfinal = [ float(i) for i in gparameters[-1] ]
        centerg, contg, fluxg, eqwg, coreg, fwhmg, fwhml = gparamfinal
        yg = gaussian(wavex,centerg,fwhmg/2.3538,coreg) + contg

        #Plot
        #sourceg = ColumnDataSource(data=dict(x=wavex,y=yg))
        p.line(wavex,yg,color='red')

show(p)

# Jan 28 15:36 irafspec-0266-51602-0051.fits - Ap 1: 
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=all
#   center      cont      flux       eqw      core     gfwhm     lfwhm
  8592.334  3.133347  0.391597    -0.125 0.0350891     10.48        0.
# Jan 28 15:36 irafspec-0266-51630-0057.fits - Ap 1: 
# Nfit=1, background=yes, positions=all, gfwhm=all, lfwhm=all
#   center      cont      flux       eqw      core     gfwhm     lfwhm
  8600.019  2.701101 -0.143245   0.05303 -0.040107     3.355        0.
